In [14]:
import json
import pandas as pd

import networkx as nx, numpy as np
from dowhy import gcm

# plus tqdm, typing_extensions, matplotlib

In [2]:
import json

with open("trace.log", 'r') as trace_file:
    trace_lines = trace_file.readlines()


trace_data = []
for trace_line in trace_lines:
    trace = json.loads(trace_line)
    trace_data.append(trace)

In [3]:
before = trace_data[:23]
after = trace_data[23:]

print(len(before), len(after))

23 16


In [4]:
nodes = [x['node']['id'] for x in trace_data[0]["profile"]] + ['totalDuration']

In [9]:
before_records = []
for trace in before:
    profile = trace["profile"]
    durations = [x["duration"] for x in profile] + [trace["totalDuration"]]
    before_records.append(durations)

before_df = pd.DataFrame(before_records, columns=nodes)


after_records = []
for trace in after:
    profile = trace["profile"]
    durations = [x["duration"] for x in profile] + [trace["totalDuration"]]
    after_records.append(durations)

after_df = pd.DataFrame(after_records, columns=nodes)

In [16]:
list()

[('beff5e93.c3a63', '2f34f44a.aca02c'),
 ('2f34f44a.aca02c', 'da968efe.e1f1f'),
 ('da968efe.e1f1f', '2f380911.38291e'),
 ('2f380911.38291e', 'e3acce5d13799473'),
 ('e3acce5d13799473', '7db5d25abf49b78e'),
 ('7db5d25abf49b78e', 'totalDuration')]

In [25]:
causal_graph = nx.DiGraph(zip(nodes[:-1], nodes[1:]))
causal_model = gcm.ProbabilisticCausalModel(causal_graph)
causal_model.set_causal_mechanism(nodes[0], gcm.EmpiricalDistribution())
for node in nodes[1:-1]:
    causal_model.set_causal_mechanism(node, gcm.EmpiricalDistribution())
causal_model.set_causal_mechanism(nodes[-1], gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))

In [26]:
attributions = gcm.distribution_change(causal_model, before_df, after_df, 'totalDuration')

/home/ubuntu/projects/causality_fbp/.venv/lib/python3.8/site-packages/dowhy/gcm/distribution_change.py:231: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joint_data = old_data.append(new_data, ignore_index=True, sort=True)


RuntimeError: Node 2f34f44a.aca02c has parents and, thus, requires a ConditionalStochasticModel, but a <dowhy.gcm.stochastic_models.EmpiricalDistribution object at 0x7fad4b574100> was found!

In [24]:
after_df

,beff5e93.c3a63,2f34f44a.aca02c,da968efe.e1f1f,2f380911.38291e,e3acce5d13799473,7db5d25abf49b78e,totalDuration
0,0,1,731,986,19,0,1737
1,0,0,689,629,16,0,1334
2,0,0,725,667,16,0,1408
3,0,0,729,663,16,0,1408
4,0,0,683,588,17,0,1288
5,0,0,749,980,16,0,1745
6,0,0,691,649,16,0,1356
7,0,0,710,1025,24,0,1759
8,0,0,697,524,16,0,1237
9,0,0,740,831,18,1,1590


In [22]:
attributions

{'2f34f44a.aca02c': 1.7354734661611006e-05,
 '2f380911.38291e': 0.0004293193403394263,
 '7db5d25abf49b78e': 8.532526291437765e-05,
 'beff5e93.c3a63': 5.900356758159357e-05,
 'da968efe.e1f1f': 6.627927290023466e-05,
 'e3acce5d13799473': -5.8509436934483134e-05,
 'totalDuration': 42.22672457516956}